<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisPredictivo/blob/master/Kaggle/2023Q2/Un_primer_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import json
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor


import pandas as pd
import matplotlib.pyplot as plt





In [14]:
df_val=pd.read_csv('dataset/testear.csv')
df_train=pd.read_csv('dataset/origen.csv')
with open("directors_score.json", "r") as read_file:
    directors_score = json.load(read_file)

In [15]:
def dir_score():
    def director_score(directors):
        score = 0
        count = 0
        for director in directors:
            if director in directors_score:
                score += directors_score[director]
                count += 1
        return score / count if count > 0 else 0

    for df in [df_train, df_val]:
        df['director_score'] = 0
        df['director_score'] = df['directors'].apply(lambda x: director_score(x.split(',')))
        df['director_score'] = df['director_score'].fillna(0)
        df['director_score'] = df['director_score'].astype(float)
        


In [16]:
def give_authors_experience(writer):
    authors = []
    if type(writer) != str:
        return authors
    
    for i,author in enumerate(writer.split(',')):
        authors.append(writer_counts[author])
    
    return sum(authors)

for df in [df_train, df_val]:

    writers = df['writers']
    writers = writers.str.split(',', expand=True)
    writer_counts = writers.stack().value_counts()
    writer_counts['0'] = 0

    df['writers_exp'] = df['writers'].apply(give_authors_experience)


del writers, writer_counts,df

In [17]:
def give_directors_experience(director):
    directors = []
    if type(director) != str:
        return directors
    
    for i,author in enumerate(director.split(',')):
        directors.append(directors_counts[author])
    
    return sum(directors)

for df in [df_train, df_val]:
        
    directors = df['directors']
    directors = directors.str.split(',', expand=True)
    directors_counts = directors.stack().value_counts()
    directors_counts['0'] = 0


    #apply the function to the writers column, send also the ratings column to the function
    df['directors_exp'] = df['directors'].apply(give_directors_experience)



del directors, directors_counts, df

In [18]:
#set language to 0 if it is nan

df_train['language'] = df_train['language'].fillna(0)
df_val['language'] = df_val['language'].fillna(0)

In [19]:
y=df_train.averageRating
vars=['startYear', 'runtimeMinutes',"numVotes","writers_exp",
      "directors_exp","endYear","isAdult","seasonNumber",
      "episodeNumber","ordering","isOriginalTitle","runtime",
      "revenue","popularity","budget"]
X=df_train[vars]
X = pd.concat([X, pd.get_dummies(df_train['language'], prefix='language')], axis=1)
X_pred=df_val[vars]
X_pred = pd.concat([X_pred, pd.get_dummies(df_val['language'], prefix='language')], axis=1)


In [20]:
for i,df in enumerate([df_train, df_val]):
    encoder = OneHotEncoder()
    title = encoder.fit_transform(df[['titleType']])
    title_df = pd.DataFrame(title.toarray(), columns=encoder.get_feature_names_out(['titleType']))
    #split by genres
    genres = df.genres_x.str.get_dummies(sep=',')
    genres = genres.reindex(sorted(genres.columns), axis=1)


    title_df = pd.concat([title_df, genres], axis=1)
    if not i:
        X = pd.concat([X, title_df], axis=1)
    else:
        X_pred = pd.concat([X_pred, title_df], axis=1)


for col in X.columns:
    if col not in X_pred.columns:
        X_pred[col] = 0

X_pred = X_pred.sort_index(axis=1)
X = X.sort_index(axis=1)
del title_df, genres, title, encoder, df





In [21]:
#drop X column named language_0
X = X.drop(columns=['language_0'])
X_pred = X_pred.drop(columns=['language_0'])
print(X.columns)

Index(['0', 'Action', 'Adult', 'Adventure', 'Animation', 'Biography', 'Comedy',
       'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'Film-Noir',
       'Game-Show', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News',
       'Reality-TV', 'Romance', 'Sci-Fi', 'Short', 'Sport', 'Talk-Show',
       'Thriller', 'War', 'Western', 'budget', 'directors_exp', 'endYear',
       'episodeNumber', 'isAdult', 'isOriginalTitle', 'language_en',
       'language_es', 'language_fr', 'language_haw', 'language_hi',
       'language_myv', 'language_yi', 'numVotes', 'ordering', 'popularity',
       'revenue', 'runtime', 'runtimeMinutes', 'seasonNumber', 'startYear',
       'titleType_movie', 'titleType_short', 'titleType_tvEpisode',
       'titleType_tvMiniSeries', 'titleType_tvMovie', 'titleType_tvSeries',
       'titleType_tvShort', 'titleType_tvSpecial', 'titleType_video',
       'titleType_videoGame', 'writers_exp'],
      dtype='object')


In [22]:
X.to_csv('dataset/train.csv', index=False)
X_pred.to_csv('dataset/val.csv', index=False)